In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def g(data):
    return pd.read_csv(StringIO(data))

In [3]:
d="""I,R,V,C,D
27,1,21,13,16
64,2,20,7,26
52,8,29,12,13
67,8,14,1,23
57,11,17,19,23
99,12,9,14,22
72,12,2,17,19
83,19,25,14,8
14,19,17,13,24
35,21,1,20,14
19,24,3,15,4
74,24,25,28,19
63,24,5,6,9
"""
df=g(d)
df.head()

,I,R,V,C,D
0,27,1,21,13,16
1,64,2,20,7,26
2,52,8,29,12,13
3,67,8,14,1,23
4,57,11,17,19,23


In [23]:
d="""A,B
Bb00,100080
Aa00,<NA>
Cc10,450089"""
df=g(d)
df['B'][1] = pd.NA
df

<ipython-input-23-803a3c9aa69d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['B'][1] = pd.NA


,A,B
0,Bb00,100080.0
1,Aa00,<NA>
2,Cc10,450089.0


In [26]:
np.where(df.B.isna(),df.A,df.B)

array([100080.0, 'Aa00', 450089.0], dtype=object)

In [30]:
d="""time,X
2007-11-30 20:20:39,6
2007-11-30 20:59:39,5
2007-11-30 21:38:38,4
2007-11-30 22:17:38,3
2007-11-30 22:56:38,2"""
df=g(d)
df = df.set_index('time')
df

,X
time,
2007-11-30 20:20:39,6
2007-11-30 20:59:39,5
2007-11-30 21:38:38,4
2007-11-30 22:17:38,3
2007-11-30 22:56:38,2


In [115]:
df = pd.DataFrame({
    'date':["DatetimeIndex(['2014-02-19'], dtype='datetime64[ns]', freq=None)",
"DatetimeIndex(['2013-02-28'], dtype='datetime64[ns]', freq=None)",
"DatetimeIndex(['2018-04-15'], dtype='datetime64[ns]', freq=None)"]
})

df

,date
0,"DatetimeIndex(['2014-02-19'], dtype='datetime6..."
1,"DatetimeIndex(['2013-02-28'], dtype='datetime6..."
2,"DatetimeIndex(['2018-04-15'], dtype='datetime6..."


In [116]:
df.date = df["date"].str.extract('\[(.*?)]', expand=True).replace("'","", regex=True)
#df.date = pd.to_datetime(df.date)
df

,date
0,2014-02-19
1,2013-02-28
2,2018-04-15


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    3 non-null      object
dtypes: object(1)
memory usage: 152.0+ bytes


In [132]:
>>> cols = pd.MultiIndex.from_tuples([("a", "b"), ("a", "c")])
>>> df = pd.DataFrame([[1,2], [3,4]], columns=cols)
>>> df


df.columns = df.columns.droplevel()
df

,b,c
0,1,2
1,3,4


In [157]:
df = pd.DataFrame({
    'test':['abc', 'DEF'],
    'num':[1,2]
})

val='Abc'

#df.query('test == @val.lower()')
df

,test,num
0,abc,1
1,DEF,2


In [156]:
pd.__version__ # '1.2.4'

'1.2.4'

In [159]:
d="""Datetime,Sample Measurement
2019-08-17 00:00:00,10
2019-08-17 01:00:00,14
2019-08-17 02:00:00,12
2019-08-17 03:00:00,10
2019-08-17 04:00:00,10
2019-08-17 05:00:00,10
2019-09-07 20:00:00,9
2019-09-07 21:00:00,7
2019-09-07 22:00:00,5
2019-09-07 23:00:00,3

"""
df=g(d)
df

,Datetime,Sample Measurement
0,2019-08-17 00:00:00,10
1,2019-08-17 01:00:00,14
2,2019-08-17 02:00:00,12
3,2019-08-17 03:00:00,10
4,2019-08-17 04:00:00,10
5,2019-08-17 05:00:00,10
6,2019-09-07 20:00:00,9
7,2019-09-07 21:00:00,7
8,2019-09-07 22:00:00,5
9,2019-09-07 23:00:00,3


In [160]:
df.Datetime = pd.to_datetime(df.Datetime)
df = df.set_index('Datetime')
df

,Sample Measurement
Datetime,
2019-08-17 00:00:00,10
2019-08-17 01:00:00,14
2019-08-17 02:00:00,12
2019-08-17 03:00:00,10
2019-08-17 04:00:00,10
2019-08-17 05:00:00,10
2019-09-07 20:00:00,9
2019-09-07 21:00:00,7
2019-09-07 22:00:00,5


In [161]:
d="""name,status,filter,new_col
A,pass,true,2
A,pass,false,0
A,fail,ture,0
B,pass,true,1
B,fail,false,0
B,pass,false,0"""
df=g(d)
df

,name,status,filter,new_col
0,A,pass,true,2
1,A,pass,false,0
2,A,fail,ture,0
3,B,pass,true,1
4,B,fail,false,0
5,B,pass,false,0


In [164]:
df.groupby(['name']).transform(lambda x: x.eq('Pass').sum()/len(x))

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
Name: status, dtype: float64

In [46]:
d="""names,date1,date2,date3,status,pass_rate
A,2004-11-23,2006-11-23,2012-11-23,pass,0
A,2008-11-23,2011-11-23,2011-11-23,pass,0
B,2015-11-23,2013-11-23,2010-11-23,pass,1
B,2013-11-23,2014-11-23,2009-11-23,fail,0
C,2012-11-23,2015-11-23,2008-11-23,pass,0
C,2021-11-23,2017-11-23,2002-11-23,pass,1
C,2009-11-23,2016-11-23,2011-11-23,pass,0"""
df=g(d)

df = df[df.groupby('names', as_index=False).apply(lambda x: (x.date1 > x.date2)  \
                         | (x.date3 < x.date1)).values]
df

,names,date1,date2,date3,status,pass_rate
2,B,2015-11-23,2013-11-23,2010-11-23,pass,1
3,B,2013-11-23,2014-11-23,2009-11-23,fail,0
4,C,2012-11-23,2015-11-23,2008-11-23,pass,0
5,C,2021-11-23,2017-11-23,2002-11-23,pass,1


In [56]:
df.groupby('names').date2.max()


names
B    2014-11-23
C    2017-11-23
Name: date2, dtype: object

In [57]:
d="""InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,Country
536370,ALARM CLOCK BAKELIKE PINK,24,2010,3.75,France
536370,ALARM CLOCK BAKELIKE Red,24,2010,3.75,France
536370,ALARM CLOCK BAKELIKE GREEN,12,2010,3.75,France
536370,PANDA AND BUNNIES STICKER SHEET,12,2010,0.85,France
536370,STARS GIFT TAPE,24,2010,0.65,France"""
df1=g(d)
df1

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,Country
0,536370,ALARM CLOCK BAKELIKE PINK,24,2010,3.75,France
1,536370,ALARM CLOCK BAKELIKE Red,24,2010,3.75,France
2,536370,ALARM CLOCK BAKELIKE GREEN,12,2010,3.75,France
3,536370,PANDA AND BUNNIES STICKER SHEET,12,2010,0.85,France
4,536370,STARS GIFT TAPE,24,2010,0.65,France


In [58]:
d="""item1,item2,item1_Support,item2_Support,Support,Confidence,Lift
ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE PINK,0.09,0.102,0.07,0.76"""
df2=g(d)
df2

,item1,item2,item1_Support,item2_Support,Support,Confidence,Lift
0,ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE PINK,0.09,0.102,0.07,0.76,NaN


In [63]:
df_m = pd.DataFrame(df2['item1'])
df_m = df_m.rename(columns={'item1':'Description'})
df_m

,Description
0,ALARM CLOCK BAKELIKE GREEN


In [65]:
d="""EXP 1,EXP 2,EXP 3,EXP 4
1,2,4,3
5,6,4,3
5,2,1,2
3,3,2,3"""
df=g(d)
df

,EXP 1,EXP 2,EXP 3,EXP 4
0,1,2,4,3
1,5,6,4,3
2,5,2,1,2
3,3,3,2,3


In [82]:
sums = df.expanding(axis=1, min_periods=2).sum().iloc[:, 1:]
sums

,EXP 2,EXP 3,EXP 4
0,3.0,7.0,10.0
1,11.0,15.0,18.0
2,7.0,8.0,10.0
3,6.0,8.0,11.0


In [83]:
sums.columns = map(lambda x: f'{df.columns[0]}-{x}', sums.columns)
sums

,EXP 1-EXP 2,EXP 1-EXP 3,EXP 1-EXP 4
0,3.0,7.0,10.0
1,11.0,15.0,18.0
2,7.0,8.0,10.0
3,6.0,8.0,11.0


In [134]:
d="""date,A,B
2021-05-19 07:00:00,Nan,Nan
2021-05-19 07:30:00,0.00,Nan
2021-05-19 08:00:00,0.00,Nan
2021-05-19 08:30:00,0.00,Nan
2021-05-19 09:00:00,19.91,Nan
2021-05-19 09:30:00,0.11,Nan
2021-05-19 10:00:00,0.00,Nan
2021-05-19 10:30:00,22.99,Nan
2021-05-19 11:00:00,0.00,Nan"""
df=g(d)
df.A = df.A.astype(float).fillna(0.)
df.loc[0, 'B']=df.loc[0,'A'].copy()
df.B = df.B.astype(float)
df.head(6)
#B1 = A1
#B2 = ((B1*5)+A2)/6

,date,A,B
0,2021-05-19 07:00:00,0.00,0.0
1,2021-05-19 07:30:00,0.00,NaN
2,2021-05-19 08:00:00,0.00,NaN
3,2021-05-19 08:30:00,0.00,NaN
4,2021-05-19 09:00:00,19.91,NaN
5,2021-05-19 09:30:00,0.11,NaN


In [135]:
df.B = ((df.B*5).shift(1)+df.A)/6
df

,date,A,B
0,2021-05-19 07:00:00,0.00,NaN
1,2021-05-19 07:30:00,0.00,0.0
2,2021-05-19 08:00:00,0.00,NaN
3,2021-05-19 08:30:00,0.00,NaN
4,2021-05-19 09:00:00,19.91,NaN
5,2021-05-19 09:30:00,0.11,NaN
6,2021-05-19 10:00:00,0.00,NaN
7,2021-05-19 10:30:00,22.99,NaN
8,2021-05-19 11:00:00,0.00,NaN


In [2]:
df = pd.DataFrame([
    ("i", 1, 'userid001'),
    ("i", 1, 'userid001'),
    ("i", 1, 'userid002'),
    ("c", 1, 'userid001'),
    ("i", 2, 'userid001'),
], columns=['type', 'cid', 'userid'])
df

,type,cid,userid
0,i,1,userid001
1,i,1,userid001
2,i,1,userid002
3,c,1,userid001
4,i,2,userid001


In [3]:
d="""ID,Type,Number_of_transactions,Amount
1,SuperMarket,8,2000
2,Hospital,2,500
1,Education,1,1000
1,Travel,3,600
1,Hospital,1,800
2,Education,1,600
1,SuperMarket,2,100
2,SuperMarket,4,400
3,SuperMarket,2,300
2,Hospital,3,200
3,Education,1,700
3,Education,1,100
2,Hospital,1,1000
3,Hotel,3,1500
3,Hotel,2,700
4,SuperMarket,10,900"""
df=g(d)
df

,ID,Type,Number_of_transactions,Amount
0,1,SuperMarket,8,2000
1,2,Hospital,2,500
2,1,Education,1,1000
3,1,Travel,3,600
4,1,Hospital,1,800
5,2,Education,1,600
6,1,SuperMarket,2,100
7,2,SuperMarket,4,400
8,3,SuperMarket,2,300
9,2,Hospital,3,200


In [10]:
df.groupby('ID').max(2)

,Number_of_transactions,Amount
ID,,
1,8,2000
2,4,1000
3,3,1500
4,10,900


In [16]:
d="""Date,CSCO,INTC,MSFT,WBA
2003-01-03,10,9,8,6
2003-01-04,9,9,6,4
2003-01-05,8,9,5,3"""
df=g(d)
df
df.Date = pd.to_datetime(df.Date)
df = df.set_index('Date')
df

,CSCO,INTC,MSFT,WBA
Date,,,,
2003-01-03,10,9,8,6
2003-01-04,9,9,6,4
2003-01-05,8,9,5,3


In [18]:
df = (df/df.shift(1)).reset_index()
df

,Date,CSCO,INTC,MSFT,WBA
0,2003-01-03,NaN,NaN,NaN,NaN
1,2003-01-04,0.900000,1.0,0.750000,0.666667
2,2003-01-05,0.888889,1.0,0.833333,0.750000


In [22]:
d="""date|product|campaign_type|total_monthly_sale
2019-01-01.|Dell|[call,email,call].|5
2019-01-01.|Apple.|[fax,fax,visit,visit].|4
2019-02-01.|Dell.|[call,fax,call]|6
2019-02-01.|Apple.|[email,email,visit].|7"""
df=pd.read_csv(StringIO(d), engine='python',sep='|')
df

,date,product,campaign_type,total_monthly_sale
0,2019-01-01.,Dell,"[call,email,call].",5
1,2019-01-01.,Apple.,"[fax,fax,visit,visit].",4
2,2019-02-01.,Dell.,"[call,fax,call]",6
3,2019-02-01.,Apple.,"[email,email,visit].",7


In [26]:
from collections import Counter
df.campaign_type.apply(Counter)

0    {'[': 1, 'c': 2, 'a': 3, 'l': 5, ',': 2, 'e': ...
1    {'[': 1, 'f': 2, 'a': 2, 'x': 2, ',': 3, 'v': ...
2    {'[': 1, 'c': 2, 'a': 3, 'l': 4, ',': 2, 'f': ...
3    {'[': 1, 'e': 2, 'm': 2, 'a': 2, 'i': 4, 'l': ...
Name: campaign_type, dtype: object

In [27]:
Counter(df["campaign_type"])

Counter({'[call,email,call].': 1,
         '[fax,fax,visit,visit].': 1,
         '[call,fax,call]': 1,
         '[email,email,visit].': 1})

In [17]:
d="""date,ts_code,close,low
2021-03-10,APPL,6.67,6.66
2021-03-10,AMA,20.24,20.12
2021-03-11,APPL,6.75,6.50
2021-03-11,AMA,21.10,20.40
2021-03-12,AMA,21.31,21.03
2021-03-12,APPL,6.81,6.76
2021-03-15,AMA,21.43,20.95
2021-03-15,APPL,6.74,6.68
2021-03-16,AMA,21.49,21.10
2021-03-16,APPL,6.74,6.67
2021-03-17,APPL,6.67,6.64
2021-03-17,AMA,21.03,20.74
2021-03-18,APPL,6.56,6.51
2021-03-18,AMA,21.56,20.84
2021-03-19,APPL,6.55,6.47
2021-03-19,AMA,20.31,20.21
2021-03-22,AMA,21.38,20.37
2021-03-22,APPL,6.63,6.55
2021-03-23,APPL,6.60,6.54
2021-03-23,AMA,21.06,20.80
2021-03-24,APPL,6.62,6.55
2021-03-24,AMA,20.37,20.29
2021-03-25,AMA,20.59,20.24
2021-03-25,APPL,6.67,6.57
2021-03-26,APPL,6.69,6.64
2021-03-26,AMA,20.98,20.60
2021-03-29,AMA,21.32,21.03
2021-03-29,APPL,6.57,6.54
2021-03-30,AMA,21.76,21.04
2021-03-30,APPL,6.42,6.40
2021-03-31,APPL,6.39,6.33
2021-03-31,AMA,21.84,21.43
2021-04-01,APPL,6.51,6.32
2021-04-01,AMA,21.61,21.33
2021-04-02,AMA,21.33,21.19
2021-04-02,APPL,6.51,6.44
2021-04-06,AMA,21.51,21.34
2021-04-06,APPL,6.60,6.50
2021-04-07,AMA,21.47,21.14
2021-04-07,APPL,6.58,6.52
2021-04-08,AMA,21.39,21.16
2021-04-08,APPL,6.43,6.40
2021-04-09,AMA,21.13,20.92
2021-04-09,APPL,6.40,6.38
2021-04-12,APPL,6.34,6.32
2021-04-12,AMA,20.54,20.47
2021-04-13,APPL,6.34,6.31
2021-04-13,AMA,20.62,20.43
2021-04-14,APPL,6.36,6.28
2021-04-14,AMA,20.51,20.26
2021-04-15,AMA,20.20,19.92
2021-04-15,APPL,6.49,6.32
2021-04-16,APPL,6.69,6.40
2021-04-16,AMA,20.10,19.66
2021-04-19,AMA,20.98,19.75
2021-04-19,APPL,6.68,6.64
2021-04-20,AMA,21.52,20.76
2021-04-20,APPL,6.68,6.64
2021-04-21,APPL,6.58,6.57
2021-04-21,AMA,22.83,22.12
2021-04-22,APPL,6.57,6.56
2021-04-22,AMA,22.80,22.60
2021-04-23,AMA,23.11,22.89
2021-04-23,APPL,6.47,6.44
2021-04-26,APPL,6.36,6.36
2021-04-26,AMA,22.76,22.72
2021-04-27,AMA,22.76,22.68
2021-04-27,APPL,6.34,6.31
2021-04-28,APPL,6.31,6.28
2021-04-28,AMA,23.17,22.60
2021-04-29,AMA,23.41,22.93
2021-04-29,APPL,6.48,6.31
2021-04-30,AMA,23.11,22.83
2021-04-30,APPL,6.33,6.30"""
df=pd.read_csv(StringIO(d))
df.date = pd.to_datetime(df.date)
df = df.set_index('date')
df

,ts_code,close,low
date,,,
2021-03-10,APPL,6.67,6.66
2021-03-10,AMA,20.24,20.12
2021-03-11,APPL,6.75,6.50
2021-03-11,AMA,21.10,20.40
2021-03-12,AMA,21.31,21.03
...,...,...,...
2021-04-28,AMA,23.17,22.60
2021-04-29,AMA,23.41,22.93
2021-04-29,APPL,6.48,6.31


In [22]:
%timeit df.groupby('ts_code').resample('W').agg({'low': 'min','close': 'last'}).reset_index().dropna()

23.6 ms ± 2.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
df.groupby(['ts_code', pd.Grouper(level=0, freq='W')]).agg({'low': 'min','close': 'last'}).reset_index().dropna()

,ts_code,date,low,close
0,AMA,2021-03-14,20.12,21.31
1,AMA,2021-03-21,20.21,20.31
2,AMA,2021-03-28,20.24,20.98
3,AMA,2021-04-04,21.03,21.33
4,AMA,2021-04-11,20.92,21.13
5,AMA,2021-04-18,19.66,20.10
6,AMA,2021-04-25,19.75,23.11
7,AMA,2021-05-02,22.60,23.11
8,APPL,2021-03-14,6.50,6.81
9,APPL,2021-03-21,6.47,6.55


In [11]:
df1 = df.groupby('ts_code').resample('W').agg({'low': 'min','close': 'last'}).reset_index().dropna()

In [12]:
df2 = df.groupby([pd.Grouper(level=0, freq='W'), 'ts_code']).agg({'low': 'min','close': 'last'}).reset_index().dropna()

In [15]:
df2

,date,ts_code,low,close
0,2021-03-14,AMA,20.12,21.31
1,2021-03-14,APPL,6.50,6.81
2,2021-03-21,AMA,20.21,20.31
3,2021-03-21,APPL,6.47,6.55
4,2021-03-28,AMA,20.24,20.98
5,2021-03-28,APPL,6.54,6.69
6,2021-04-04,AMA,21.03,21.33
7,2021-04-04,APPL,6.32,6.51
8,2021-04-11,AMA,20.92,21.13
9,2021-04-11,APPL,6.38,6.40


In [30]:
d="""DE,SW,Tech,COUNT(*)
2021-01-01,s1,t1,1
2021-04-02,s1,t1,699
2021-04-02,s1,t2,268
2021-04-06,s1,t1,2
2021-04-06,s2,t1,466
2021-04-06,s2,t2,81
2021-04-06,s2,t3,954"""
df=g(d)
df

,DE,SW,Tech,COUNT(*)
0,2021-01-01,s1,t1,1
1,2021-04-02,s1,t1,699
2,2021-04-02,s1,t2,268
3,2021-04-06,s1,t1,2
4,2021-04-06,s2,t1,466
5,2021-04-06,s2,t2,81
6,2021-04-06,s2,t3,954


In [4]:
d="""date,ts_code,close,low
2021-03-10,APPL,6.67,6.66
2021-03-10,AMA,20.24,20.12
2021-03-11,APPL,6.75,6.50
2021-03-11,AMA,21.10,20.40
2021-03-12,AMA,21.31,21.03
2021-03-12,APPL,6.81,6.76
2021-03-15,AMA,21.43,20.95
2021-03-15,APPL,6.74,6.68
2021-03-16,AMA,21.49,21.10
2021-03-16,APPL,6.74,6.67
2021-03-17,APPL,6.67,6.64
2021-03-17,AMA,21.03,20.74
2021-03-18,APPL,6.56,6.51
2021-03-18,AMA,21.56,20.84
2021-03-19,APPL,6.55,6.47
2021-03-19,AMA,20.31,20.21
2021-03-22,AMA,21.38,20.37
2021-03-22,APPL,6.63,6.55
2021-03-23,APPL,6.60,6.54
2021-03-23,AMA,21.06,20.80
2021-03-24,APPL,6.62,6.55
2021-03-24,AMA,20.37,20.29
2021-03-25,AMA,20.59,20.24
2021-03-25,APPL,6.67,6.57
2021-03-26,APPL,6.69,6.64
2021-03-26,AMA,20.98,20.60
2021-03-29,AMA,21.32,21.03
2021-03-29,APPL,6.57,6.54
2021-03-30,AMA,21.76,21.04
2021-03-30,APPL,6.42,6.40
2021-03-31,APPL,6.39,6.33
2021-03-31,AMA,21.84,21.43
2021-04-01,APPL,6.51,6.32
2021-04-01,AMA,21.61,21.33
2021-04-02,AMA,21.33,21.19
2021-04-02,APPL,6.51,6.44
2021-04-06,AMA,21.51,21.34
2021-04-06,APPL,6.60,6.50
2021-04-07,AMA,21.47,21.14
2021-04-07,APPL,6.58,6.52
2021-04-08,AMA,21.39,21.16
2021-04-08,APPL,6.43,6.40
2021-04-09,AMA,21.13,20.92
2021-04-09,APPL,6.40,6.38
2021-04-12,APPL,6.34,6.32
2021-04-12,AMA,20.54,20.47
2021-04-13,APPL,6.34,6.31
2021-04-13,AMA,20.62,20.43
2021-04-14,APPL,6.36,6.28
2021-04-14,AMA,20.51,20.26
2021-04-15,AMA,20.20,19.92
2021-04-15,APPL,6.49,6.32
2021-04-16,APPL,6.69,6.40
2021-04-16,AMA,20.10,19.66
2021-04-19,AMA,20.98,19.75
2021-04-19,APPL,6.68,6.64
2021-04-20,AMA,21.52,20.76
2021-04-20,APPL,6.68,6.64
2021-04-21,APPL,6.58,6.57
2021-04-21,AMA,22.83,22.12
2021-04-22,APPL,6.57,6.56
2021-04-22,AMA,22.80,22.60
2021-04-23,AMA,23.11,22.89
2021-04-23,APPL,6.47,6.44
2021-04-26,APPL,6.36,6.36
2021-04-26,AMA,22.76,22.72
2021-04-27,AMA,22.76,22.68
2021-04-27,APPL,6.34,6.31
2021-04-28,APPL,6.31,6.28
2021-04-28,AMA,23.17,22.60
2021-04-29,AMA,23.41,22.93
2021-04-29,APPL,6.48,6.31
2021-04-30,AMA,23.11,22.83
2021-04-30,APPL,6.33,6.30
2021-07-30,APPL,6.33,6.30"""
df=pd.read_csv(StringIO(d))
df.date = pd.to_datetime(df.date)
df = df.set_index('date')
df

,ts_code,close,low
date,,,
2021-03-10,APPL,6.67,6.66
2021-03-10,AMA,20.24,20.12
2021-03-11,APPL,6.75,6.50
2021-03-11,AMA,21.10,20.40
2021-03-12,AMA,21.31,21.03
...,...,...,...
2021-04-29,AMA,23.41,22.93
2021-04-29,APPL,6.48,6.31
2021-04-30,AMA,23.11,22.83


In [5]:
df.groupby('ts_code').resample('W').agg({'low': 'min','close': 'last'})

low  close
ts_code date                    
AMA     2021-03-14  20.12  21.31
        2021-03-21  20.21  20.31
        2021-03-28  20.24  20.98
        2021-04-04  21.03  21.33
        2021-04-11  20.92  21.13
        2021-04-18  19.66  20.10
        2021-04-25  19.75  23.11
        2021-05-02  22.60  23.11
APPL    2021-03-14   6.50   6.81
        2021-03-21   6.47   6.55
        2021-03-28   6.54   6.69
        2021-04-04   6.32   6.51
        2021-04-11   6.38   6.40
        2021-04-18   6.28   6.69
        2021-04-25   6.44   6.47
        2021-05-02   6.28   6.33
        2021-05-09    NaN    NaN
        2021-05-16    NaN    NaN
        2021-05-23    NaN    NaN
        2021-05-30    NaN    NaN
        2021-06-06    NaN    NaN
        2021-06-13    NaN    NaN
        2021-06-20    NaN    NaN
        2021-06-27    NaN    NaN
        2021-07-04    NaN    NaN
        2021-07-11    NaN    NaN
        2021-07-18    NaN    NaN
        2021-07-25    NaN    NaN
        2021-08-01   6.30   6.33

In [6]:
df.groupby(['ts_code', pd.Grouper(level=0, freq='W')]).agg({'low': 'min','close': 'last'}).reset_index().dropna()

,ts_code,date,low,close
0,AMA,2021-03-14,20.12,21.31
1,AMA,2021-03-21,20.21,20.31
2,AMA,2021-03-28,20.24,20.98
3,AMA,2021-04-04,21.03,21.33
4,AMA,2021-04-11,20.92,21.13
5,AMA,2021-04-18,19.66,20.10
6,AMA,2021-04-25,19.75,23.11
7,AMA,2021-05-02,22.60,23.11
8,APPL,2021-03-14,6.50,6.81
9,APPL,2021-03-21,6.47,6.55


In [9]:
d="""date,ts_code,close,low
2021-04-29,AMA,23.41,22.93
2021-04-29,APPL,6.48,6.31
2021-04-30,AMA,23.11,22.83
2021-04-30,APPL,6.33,6.30
2021-05-30,APPL,6.33,6.30"""
df = pd.read_csv(StringIO(d))
df.date = pd.to_datetime(df.date)
df = df.set_index('date')
df

,ts_code,close,low
date,,,
2021-04-29,AMA,23.41,22.93
2021-04-29,APPL,6.48,6.31
2021-04-30,AMA,23.11,22.83
2021-04-30,APPL,6.33,6.30
2021-05-30,APPL,6.33,6.30


In [11]:
df.groupby('ts_code').resample('W').agg({'low': 'min','close': 'last'}).reset_index()

,ts_code,date,low,close
0,AMA,2021-05-02,22.83,23.11
1,APPL,2021-05-02,6.30,6.33
2,APPL,2021-05-09,NaN,NaN
3,APPL,2021-05-16,NaN,NaN
4,APPL,2021-05-23,NaN,NaN
5,APPL,2021-05-30,6.30,6.33


In [12]:
df.groupby(['ts_code', pd.Grouper(level=0, freq='W')]).agg({'low': 'min','close': 'last'}).reset_index()

,ts_code,date,low,close
0,AMA,2021-05-02,22.83,23.11
1,APPL,2021-05-02,6.30,6.33
2,APPL,2021-05-30,6.30,6.33
